In [1]:
from os.path import dirname, basename

import pandas as pd

from inpainting.load import load_dataframe
from inpainting.utils import get_paths

In [2]:
dataset = 'demo'
results_dirs = get_paths(f'../results/ts/{dataset}/*')

times_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Benchmark/*/inference_times.csv'):
        times_df = load_dataframe(path)
        times_df['model'] = basename(results_dir)
        times_df['sequence_name'] = basename(dirname(path))
        times_dfs.append(times_df)
times_df = pd.concat(times_dfs)

results_dfs = []
for results_dir in results_dirs:
    for path in get_paths(f'{results_dir}/Evaluation/*/results.csv'):
        results_df = load_dataframe(path)
        results_df['model'] = basename(results_dir)
        sequence_name = basename(dirname(path))
        results_df['sequence_name'] = sequence_name
        results_df['object_sequence_name'] = sequence_name.split('_')[1]
        results_df['object_id'] = int(sequence_name.split('_')[2])
        results_dfs.append(results_df)
results_df = pd.concat(results_dfs)

df = pd.merge(times_df, results_df, on=['model', 'sequence_name', 'frame_id'])
df

,frame_id,inference_time,model,sequence_name,object_coverage,background_coverage,object_sequence_name,object_id
0,0,29.907040,ts_dilation_3_3,breakdance_breakdance_001,0.909841,0.962765,breakdance,1
1,1,30.807487,ts_dilation_3_3,breakdance_breakdance_001,0.905556,0.972513,breakdance,1
2,2,29.123968,ts_dilation_3_3,breakdance_breakdance_001,0.862681,0.972409,breakdance,1
3,3,25.617023,ts_dilation_3_3,breakdance_breakdance_001,0.862861,0.973241,breakdance,1
4,4,27.912607,ts_dilation_3_3,breakdance_breakdance_001,0.844502,0.980657,breakdance,1
...,...,...,...,...,...,...,...,...
1335,64,19.942720,ts_no_dilation,tennis_tennis_001,0.917388,0.992872,tennis,1
1336,65,20.285055,ts_no_dilation,tennis_tennis_001,0.949545,0.991628,tennis,1
1337,66,19.665312,ts_no_dilation,tennis_tennis_001,0.944939,0.992127,tennis,1
1338,67,18.892609,ts_no_dilation,tennis_tennis_001,0.937358,0.993678,tennis,1


In [3]:
metrics = pd.DataFrame()
metrics['model'] = df['model']
metrics['sequence_name'] = df['sequence_name']
metrics['fps'] = df['inference_time']
metrics['object_coverage'] = df['object_coverage']
metrics['background_coverage'] = df['background_coverage']
metrics.groupby('model').mean()

,fps,object_coverage,background_coverage
model,,,
ts_dilation_3_3,21.848510,0.636011,0.971877
ts_dilation_3_5,27.034036,0.643060,0.964121
ts_dilation_5_3,24.047414,0.645923,0.960203
ts_dilation_5_5,29.392590,0.655772,0.944328
ts_no_dilation,20.881209,0.613708,0.982910
